In [16]:
import pickle

filename = "collabRecommendation.pickle"


In [17]:
# load model
model_knn = pickle.load(open(filename, "rb"))

In [18]:
print(model_knn)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)


In [19]:
import csv
# load hashmap
hashmap = {}
with open('itemBasedRecommendation_hashmap_1.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for rows in reader:
        hashmap[rows[0]] = int(rows[1])

game_names = {}
with open('review_counts.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for rows in reader:
        game_names[rows[0]] = rows[1]

In [14]:
from scipy.sparse import csr_matrix
# convert dataframe of movie features to scipy sparse matrix
# load from csv into dataframe
import pandas as pd
# df_movie_features = pd.read_csv('game_features.csv', index_col=0)
with open('df_movie_features.pkl', mode='rb') as infile:
    df_movie_features = pickle.load(infile)
mat_movie_features = csr_matrix(df_movie_features.values)

In [24]:
n_recommendations = 10
game_id = "105600"

# get input movie index
# inference
print('Recommendation system start to make inference')
print('......\n')
distances, indices = model_knn.kneighbors(
    mat_movie_features[hashmap[game_id]],
    n_neighbors=n_recommendations+1)
# get list of raw idx of recommendations
raw_recommends = \
    sorted(
        list(
            zip(
                indices.squeeze().tolist(),
                distances.squeeze().tolist()
            )
        ),
        key=lambda x: x[1]
    )[:0:-1]
# print('It took my system {:.2f}s to make inference \n\
#         '.format(time.time() - t0))

reverse_hashmap = {v: k for k, v in hashmap.items()}
# print('Recommendations for {}:'.format(fav_movie))
recommended_games = []
for i, (idx, dist) in enumerate(raw_recommends):
    recommend_game_id = str(reverse_hashmap[idx])
    recommended_games.append({'game_id': recommend_game_id, 'rank': i+1, 'score': round(dist*10000 - 9900, 1), 'game_name': game_names[recommend_game_id]})
    # print('{0}: {1}, with distance '
    #         'of {2}'.format(i+1, reverse_hashmap[idx], dist))

Recommendation system start to make inference
......



In [25]:
print(recommended_games)

[{'game_id': '367520', 'rank': 1, 'score': 58.1, 'game_name': 'Hollow Knight'}, {'game_id': '391540', 'rank': 2, 'score': 57.3, 'game_name': 'Undertale'}, {'game_id': '250900', 'rank': 3, 'score': 57.0, 'game_name': 'The Binding of Isaac: Rebirth'}, {'game_id': '218620', 'rank': 4, 'score': 53.4, 'game_name': 'PAYDAY 2'}, {'game_id': '204360', 'rank': 5, 'score': 52.9, 'game_name': 'Castle Crashers'}, {'game_id': '620', 'rank': 6, 'score': 48.3, 'game_name': 'Portal 2'}, {'game_id': '945360', 'rank': 7, 'score': 48.2, 'game_name': 'Among Us'}, {'game_id': '322330', 'rank': 8, 'score': 45.3, 'game_name': "Don't Starve Together"}, {'game_id': '413150', 'rank': 9, 'score': 41.1, 'game_name': 'Stardew Valley'}, {'game_id': '4000', 'rank': 10, 'score': 33.4, 'game_name': "Garry's Mod"}]
